In [1]:
import torch
import torchvision
from torch.optim import Adam

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
train_data = torchvision.datasets.FashionMNIST("./data/", train=True, transform=torchvision.transforms.ToTensor(), download=True)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=10, shuffle=True)

In [4]:
test_data = torchvision.datasets.FashionMNIST("./data/", transform=torchvision.transforms.ToTensor(), download=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=10, shuffle=True)

In [10]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = torch.nn.Conv2d(1, 16, 5, padding=2)
        self.conv2 = torch.nn.Conv2d(16, 32, 5, padding=2)
        self.act = torch.functional.F.relu
        self.bn1 = torch.nn.BatchNorm2d(16)
        self.bn2 = torch.nn.BatchNorm2d(32)
        self.pool = torch.nn.MaxPool2d(2,2)
        self.fc1 = torch.nn.Linear(49*32, 600)
        self.fc1_drop = torch.nn.Dropout(p=0.4)
        self.fc2 = torch.nn.Linear(600, 10)
        
    def forward(self, x):
        x = self.pool(self.act(self.bn1(self.conv1(x))))
        x = self.pool(self.act(self.bn2(self.conv2(x))))
        x = x.view(x.size(0), -1)
        x = self.act(self.fc1(x))
        x = self.fc1_drop(x)
        x = self.fc2(x)
        return x
    
net = Net()
net.to(device)

Net(
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1568, out_features=600, bias=True)
  (fc1_drop): Dropout(p=0.4, inplace=False)
  (fc2): Linear(in_features=600, out_features=10, bias=True)
)

In [11]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = Adam(net.parameters(), lr=0.001)

In [12]:
avg_acc = []
test_acc = []

for epoch in range(15):
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        
        result = net(images)
        loss = criterion(result, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if i%1000==0:
            print("Epoch {}\tBatch {}\tAvg loss {}".format(epoch, i, running_loss/1000))
            running_loss = 0.
            
    

Epoch 0	Batch 0	Avg loss 0.00227469277381897
Epoch 0	Batch 1000	Avg loss 0.6677938705049455
Epoch 0	Batch 2000	Avg loss 0.4537099851053208
Epoch 0	Batch 3000	Avg loss 0.41379839173331856
Epoch 0	Batch 4000	Avg loss 0.3687911455221474
Epoch 0	Batch 5000	Avg loss 0.35322407259326427
Epoch 1	Batch 0	Avg loss 0.0002069956064224243
Epoch 1	Batch 1000	Avg loss 0.312706984161865
Epoch 1	Batch 2000	Avg loss 0.3057110340252984
Epoch 1	Batch 3000	Avg loss 0.3011418948993087
Epoch 1	Batch 4000	Avg loss 0.29454160649422556
Epoch 1	Batch 5000	Avg loss 0.2944466896709055
Epoch 2	Batch 0	Avg loss 0.00030434852838516236
Epoch 2	Batch 1000	Avg loss 0.26082843192172006
Epoch 2	Batch 2000	Avg loss 0.25289026927569647
Epoch 2	Batch 3000	Avg loss 0.24813144108647248
Epoch 2	Batch 4000	Avg loss 0.26281130335037595
Epoch 2	Batch 5000	Avg loss 0.25187697855115404
Epoch 3	Batch 0	Avg loss 0.0006295552849769592
Epoch 3	Batch 1000	Avg loss 0.22711684453912312
Epoch 3	Batch 2000	Avg loss 0.22272300871532935
Epoch

In [13]:
correct = 0
total = 0

with torch.no_grad():
    for i, data in enumerate(test_loader):
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)

        correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)
    
print("Total accuracy is {}%".format(round(correct/total*100, 2)))

Total accuracy is 96.35%


In [14]:
class_correct = list(0. for i in range(len(train_data.classes)))
class_total = list(0. for i in range(len(train_data.classes)))

In [15]:
with torch.no_grad():
    for i, data in enumerate(test_loader):
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels)
        for i in range(10):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1
            
for i in range(10):
    print('Accuracy of %15s : %2d %%' % (
        train_data.classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of     T-shirt/top : 91 %
Accuracy of         Trouser : 99 %
Accuracy of        Pullover : 92 %
Accuracy of           Dress : 96 %
Accuracy of            Coat : 92 %
Accuracy of          Sandal : 99 %
Accuracy of           Shirt : 94 %
Accuracy of         Sneaker : 99 %
Accuracy of             Bag : 99 %
Accuracy of      Ankle boot : 98 %
